<span class='note'>*Make me look good.* Click on the cell below and press <kbd>Ctrl</kbd>-<kbd>Enter</kbd>.</span>

In [ ]:
from IPython.core.display import HTML
HTML(open('css/custom.css', 'r').read())

<h5 class='prehead'>SA367 &middot; Mathematical Models for Decision Making &middot; Spring 2021 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 16.</h5>

<h1 class='lesson_title'>Solving the Points-After-Touchdown Problem</h1>

* Let's solve the stochastic dynamic program we formulated for the points-after-touchdown problem in Lesson 14.

- Before doing anything else, let's import `StochasticDP` from the `stochasticdp` package:

In [ ]:
from stochasticdp import StochasticDP

## Setting up the data

* In Lesson 14, we worked with the following data:

\begin{alignat*}{2}
  T & = \text{total number of possessions} \\[1ex]
  p_{1n} & = \Pr\{ \text{1-pt. conv. successful for Team $n$} \,|\, \text{1-pt. conv. attempted by Team $n$} \} & \quad & \text{for } n = \text{A}, \text{B} \\
  p_{2n} & = \Pr\{ \text{2-pt. conv. successful for Team $n$} \,|\, \text{2-pt. conv. attempted by Team $n$} \} & \quad & \text{for } n = \text{A}, \text{B} \\[1ex]
  b_1 & = \Pr \{ \text{1-pt. conv. attempted by Team B} \}\\
  b_2 & = \Pr \{ \text{2-pt. conv. attempted by Team B} \}\\[1ex]
  t_n & = \Pr \{ \text{TD by Team $n$ in 1 possession} \}                                                                     & \quad & \text{for } n = \text{A}, \text{B} \\
  g_n & = \Pr \{ \text{FG by Team $n$ in 1 possession} \}                                                                     & \quad & \text{for } n = \text{A}, \text{B} \\
  z_n & = \Pr \{ \text{no score by Team $n$ in 1 possession} \}                                                               & \quad & \text{for } n = \text{A}, \text{B} \\[1ex]
  r & = \Pr \{ \text{Team A wins in overtime} \}
\end{alignat*}

* Let's begin by defining numerical values for this data.

* We can find most of these values from [Pro Football Reference](http://www.pro-football-reference.com/years/2014/).

* For now, let's assume that Team A and Team B are both average 2014 NFL teams.
    - Recall that in 2014, 1-pt. conversions started at the 2-yard line.

* Also, let's assume that Team A wins in overtime with probability 0.5.

In [ ]:
# Total number of possessions
# Drive Averages: 2 * (#Dr) / (G * (# of teams))
T = 23

# 1-pt. conversion success probabilities
# Kicking and Punting: XP%
p1A = 0.993
p1B = 0.993

# 2-pt. conversion success probabilities
# Scoring Offense: 2PM / 2PA
p2A = 0.483
p2B = 0.483

# 1-pt. vs 2-pt attempts
# 1-pt.: Scoring Offense: XPA / (XPA + 2PA)
b1 = 0.954
b2 = 1 - b1

# Possession outcome probabilities - Team A
# TD: (Scoring Offense: ATD) / (Drive Averages: #Dr)
# FG: (Scoring Offense: FGM) / (Drive Averages: #Dr)
tA = 0.218
gA = 0.172
zA = 1 - tA - gA

# Possession outcome probabilities - Team B
tB = 0.218
gB = 0.172
zB = 1 - tB - gB

# Probability that Team A wins in OT
r = 0.5

## Initializing the stochastic dynamic program

* Stages:

\begin{align*}
  t = 0, 1, \dots, T - 1 \quad & \leftrightarrow \quad \text{end of possession $t$} \\
  t = T \quad                  & \leftrightarrow \quad \text{end of game}
\end{align*}

In [ ]:
# Number of stages


In [ ]:
# Solutions
# Number of stages
number_of_stages = T + 1

* States:

\begin{alignat*}{2}
  (n, k, d) \quad \leftrightarrow \quad & \text{Team $n$'s possession just ended} & \quad & \text{for } n \in \{\text{A}, \text{B}\}      \\
                                        & \text{Team $n$ just scored $k$ points}  & \quad & \text{for } k \in \{0, 3, 6\}                 \\
                                        & \text{Team A is ahead by $d$ points}    & \quad & \text{for } d \in \{\dots, -1, 0, 1, \dots,\}
\end{alignat*}

* In Lesson 14, we did not assume a lower or upper bound on $d$, the values that represent Team A's lead. 

* Since we need to have a finite number of states, let's assume $-20 \le d \le 20$.

* Some Python reminders:
    * We can construct a list by 
        - first creating an empty list,
        - and then adding items to it using `.append()`. 
    * For example:
        ```python
        my_list = []
        for i in range(10):
            my_list.append(i)
        ```
    * `range(a)` iterates over the integers `0, 1, ..., a - 1`, while `range(a, b)` iterates over the integers `a, a + 1, ..., b - 1`.

In [ ]:
# List of states


In [ ]:
# Solutions
# List of states
states = []
for n in ['A', 'B']:
    for k in [0, 3, 6]:
        for d in range(-20, 21):
            states.append((n, k, d))

* _Quick check._ Let's inspect what we just created:

In [ ]:
# Print the list of states
print(states)

* Allowable decisions $x_t$ at stage $t$ and state $(n, k, d)$:

\begin{alignat*}{2}
  x_t & \in \{ 1, 2 \} & \quad & \text{if } n = \text{A} \text{ and } k = 6             \\
  x_t & = \text{none}  & \quad & \text{if } n = \text{A} \text{ and } k \in \{0, 3\}    \\
  x_t & = \text{none}  & \quad & \text{if } n = \text{B} \text{ and } k \in \{0, 3, 6\}
\end{alignat*}

In [ ]:
# List of decisions


In [ ]:
# Solutions
# List of decisions
decisions = [1, 2, 'none']

* Now we can initialize a `StochasticDP` object called `dp` as follows:

In [ ]:
# Initialize stochastic dynamic program - we want to maximize, so minimize = False
dp = StochasticDP(number_of_stages, states, decisions, minimize=False)

## Transition probabilities from stages $t = 0, 1, \dots, T-2$

* Let's start with the easier transitions and work our way up to the more complicated ones.

### From states $(A, 3, d)$

* Let's consider the transitions from states $(A, 3, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

<img src="img/a3d-1.png" width="80%"/>

* We can put in these transitions like this:

In [ ]:
# Transition probabilities from (A, 3, d) up to stage T - 2


In [ ]:
# Solutions
# Transition probabilities from (A, 3, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        dp.add_transition(t, ('A', 3, d), 'none', ('B', 6, d - 6), tB, 0)
        dp.add_transition(t, ('A', 3, d), 'none', ('B', 3, d - 3), gB, 0)
        dp.add_transition(t, ('A', 3, d), 'none', ('B', 0, d), zB, 0)    

* Why are we getting an `'Invalid state, stage, or decision'` message?

    * Remember that in Lesson 14, we assumed that $d$ could take on an infinite number values.

    * On the other hand, here, we have limited $d$ to be between $-20$ and $20$.

* How can we work around this?

    * Let's do this: if $d$ is supposed to be less than $-20$, then we simply assume that it is the same as having $d = -20$.

    * For example, suppose $d = -16$ in the diagram above. Then we can model the transitions like this:

<img src="img/a3d-2.png" width="80%"/>

* So... let's start over:

In [ ]:
# Initialize stochastic dynamic program - we want to maximize, so minimize = False
dp = StochasticDP(number_of_stages, states, decisions, minimize=False)

* Using the idea above, we can write the following code to represent the transitions from states $(A, 3, d)$ for $d = -20, \dots, 20$ and $t = 0, 1, \dots, T - 2$:

In [ ]:
# Transition probabilities from (A, 3, d) up to stage T - 2


In [ ]:
# Solutions
# Transition probabilities from (A, 3, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        dp.add_transition(t, ('A', 3, d), 'none', ('B', 6, max(d - 6, -20)), tB, 0)
        dp.add_transition(t, ('A', 3, d), 'none', ('B', 3, max(d - 3, -20)), gB, 0)
        dp.add_transition(t, ('A', 3, d), 'none', ('B', 0, d), zB, 0)        

### From states $(A, 0, d)$

* Similarly, for the transitions from states $(A, 0, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

<img src="img/a0d.png" width="80%"/>

In [ ]:
# Transition probabilities from (A, 0, d) up to stage T - 2


In [ ]:
# Solutions
# Transition probabilities from (A, 0, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        dp.add_transition(t, ('A', 0, d), 'none', ('B', 6, max(d - 6, -20)), tB, 0)
        dp.add_transition(t, ('A', 0, d), 'none', ('B', 3, max(d - 3, -20)), gB, 0)
        dp.add_transition(t, ('A', 0, d), 'none', ('B', 0, d), zB, 0)        

### From states $(B, 3, d)$

* Next, the transitions from states $(B, 3, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

<img src="img/b3d-1.png" width="80%"/>

* We run into a similar problem: if we're not careful, we can end up with values of $d$ greater than 20.

* Let's do this: if $d$ is supposed to be greater than $20$, then we simply assume that it is the same as having $d = 20$, like this: 

<img src="img/b3d-2.png" width="80%"/>

In [ ]:
# Transition probabilities from (B, 3, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        dp.add_transition(t, ('B', 3, d), 'none', ('A', 6, min(d + 6, 20)), tA, 0)
        dp.add_transition(t, ('B', 3, d), 'none', ('A', 3, min(d + 3, 20)), gA, 0)
        dp.add_transition(t, ('B', 3, d), 'none', ('A', 0, d), zA, 0)

### From states $(B, 0, d)$

* Next, the transitions from states $(B, 0, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

<img src="img/b0d.png" width="80%"/>

In [ ]:
# Transition probabilities from (B, 0, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        dp.add_transition(t, ('B', 0, d), 'none', ('A', 6, min(d + 6, 20)), tA, 0)
        dp.add_transition(t, ('B', 0, d), 'none', ('A', 3, min(d + 3, 20)), gA, 0)
        dp.add_transition(t, ('B', 0, d), 'none', ('A', 0, d), zA, 0)

### From states $(A, 6, d)$

* Next up: transitions from the state $(A, 6, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

<img src="img/a6d-1.png" width="80%"/>

* In this case, we can also have values of $d$ lower than $-20$ or higher than $20$ if we're not careful.

* Because of this, we can also have multiple transitions to the same state, like this:

<img src="img/a6d-2.png" width="80%"/>

* To take care of this, we need to merge these transitions.

* The code becomes a bit more complicated, though, since we need to consider various cases.

* First, let's consider the 1-point conversion decision and break down the cases depending on the value of $d$:

In [ ]:
# Remember: 
# dp.add_transition(stage, from_state, decision, to_state, probability, contribution)

# Transition probabilities from (A, 6, d) up to stage T - 2
# Decision: 1-point conversion
for t in range(T - 1):
    for d in range(-20, 21):

        if d - 5 <= -20:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 6, -20), (1 - p1A) * tB + p1A * tB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 6, max(d - 6, -20)), (1 - p1A) * tB, 0)
            dp.add_transition(t, ('A', 6, d), 1, ('B', 6, max(d - 5, -20)), p1A * tB, 0)
        
        if d - 2 <= -20:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 3, -20), (1 - p1A) * gB + p1A * gB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 3, max(d - 3, -20)), (1 - p1A) * gB, 0)
            dp.add_transition(t, ('A', 6, d), 1, ('B', 3, max(d - 2, -20)), p1A * gB, 0)
        
        if d >= 20:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 0, 20), (1 - p1A) * zB + p1A * zB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 0, min(d, 20)), (1 - p1A) * zB, 0)
            dp.add_transition(t, ('A', 6, d), 1, ('B', 0, min(d + 1, 20)), p1A * zB, 0)

* Let's do the same for the 2-point conversion decision:

In [ ]:
# Remember: 
# dp.add_transition(stage, from_state, decision, to_state, probability, contribution)

# Transition probabilities from (A, 6, d) up to stage T - 2
# Decision: 2-point conversion
for t in range(T - 1):
    for d in range(-20, 21):

        if d - 4 <= -20:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 6, -20), (1 - p2A) * tB + p2A * tB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 6, max(d - 6, -20)), (1 - p2A) * tB, 0)
            dp.add_transition(t, ('A', 6, d), 2, ('B', 6, max(d - 4, -20)), p2A * tB, 0)
        
        if d - 1 <= -20:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 3, -20), (1 - p2A) * gB + p2A * gB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 3, max(d - 3, -20)), (1 - p2A) * gB, 0)
            dp.add_transition(t, ('A', 6, d), 2, ('B', 3, max(d - 1, -20)), p2A * gB, 0)
            
        if d >= 20:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 0, 20),  (1 - p2A) * zB + p2A * zB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 0, min(d, 20)), (1 - p2A) * zB, 0)
            dp.add_transition(t, ('A', 6, d), 2, ('B', 0, min(d + 2, 20)), p2A * zB, 0)

### From states $(B, 6, d)$

* We need to take similar care for the transitions from states $(B, 6, d)$ for $d = -20, \dots, 20$ in stages $t = 0, 1, \dots, T-2$:

<img src="img/b6d.png" width="80%"/>

In [ ]:
# Transition probabilities from (B, 6, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        if d + 4 >= 20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, 20), (1 - b1*p1B - b2*p2B) * tA + b1*p1B*tA + b2*p2B*tA, 0)
        elif d + 5 >= 20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, 20), (1 - b1*p1B - b2*p2B) * tA + b1*p1B*tA, 0)    
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, min(d + 4, 20)), b2*p2B*tA, 0)
        else:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, min(d + 6, 20)), (1 - b1*p1B - b2*p2B) * tA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, min(d + 5, 20)), b1*p1B*tA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, min(d + 4, 20)), b2*p2B*tA, 0)

        if d + 1 >= 20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, 20), (1 - b1*p1B - b2*p2B) * gA + b1*p1B*gA + b2*p2B*gA, 0)
        elif d + 2 >= 20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, 20), (1 - b1*p1B - b2*p2B) * gA + b1*p1B*gA, 0)    
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, min(d + 1, 20)), b2*p2B*gA, 0)
        else:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, min(d + 3, 20)), (1 - b1*p1B - b2*p2B) * gA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, min(d + 2, 20)), b1*p1B*gA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, min(d + 1, 20)), b2*p2B*gA, 0)
            
        if d <= -20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, -20), (1 - b1*p1B - b2*p2B) * zA + b1*p1B*zA + b2*p2B*zA, 0)
        elif d - 1 <= -20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, d), (1 - b1*p1B - b2*p2B) * zA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, -20), b1*p1B*zA + b2*p2B*zA, 0)    
        else:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, d), (1 - b1*p1B - b2*p2B) * zA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, max(d - 1, -20)), b1*p1B*zA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, max(d - 2, - 20)), b2*p2B*zA, 0)

## Transition probabilities from stage $T - 1$

* Now, we can tackle the transitions from stage $T-1$.

* From states $(A, 6, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

<center><img src="img/last_a6d.png" width="80%"></center>

In [ ]:
# Transition probabilities from (A, 6, d) in stage T - 1
for d in range(-20, 21):
    # 1-point conversion
    if d >= 20:
        dp.add_transition(T - 1, ('A', 6, d), 1, ('A', 6, 20), (1 - p1A) + p1A, 0)
    else:
        dp.add_transition(T - 1, ('A', 6, d), 1, ('A', 6, d), 1 - p1A, 0)
        dp.add_transition(T - 1, ('A', 6, d), 1, ('A', 6, min(d + 1, 20)), p1A, 0)

    # 2-point conversion
    if d >= 20:
        dp.add_transition(T - 1, ('A', 6, d), 2, ('A', 6, 20), (1 - p2A) + p2A, 0)
    else:
        dp.add_transition(T - 1, ('A', 6, d), 2, ('A', 6, d), 1 - p2A, 0)
        dp.add_transition(T - 1, ('A', 6, d), 2, ('A', 6, min(d + 2, 20)), p2A, 0)

* From states $(A, 3, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

<center><img src="img/last_a3d.png" width="70%"></center>

In [ ]:
# Transition probabilities from (A, 3, d) in stage T - 1
for d in range(-20, 21):
    dp.add_transition(T - 1, ('A', 3, d), 'none', ('A', 3, d), 1, 0)

* From states $(A, 0, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

<center><img src="img/last_a0d.png" width="70%"></center>

In [ ]:
# Transition probabilities from (A, 0, d) in stage T - 1
for d in range(-20, 21):
    dp.add_transition(T - 1, ('A', 0, d), 'none', ('A', 0, d), 1, 0)

* From states $(B, 6, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

<center><img src="img/last_b6d.png" width="80%"></center>

In [ ]:
# Transition probabilities from (B, 6, d) in stage T - 1
for d in range(-20, 21):
    if d <= -20:
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, -20), 1, 0)
    elif d <= -19:
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, -20), b1*p1B + b2*p2B, 0)
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, d), 1 - b1*p1B - b2*p2B, 0)
    else:        
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, max(d - 2, -20)), b2*p2B, 0)
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, max(d - 1, -20)), b1*p1B, 0)
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, d), 1 - b1*p1B - b2*p2B, 0)

* From states $(B, 3, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

<center><img src="img/last_b3d.png" width="70%"></center>

In [ ]:
# Transition probabilities from (B, 3, d) in stage T - 1
for d in range(-20, 21):
    dp.add_transition(T - 1, ('B', 3, d), 'none', ('B', 3, d), 1, 0)

* From states $(B, 0, d)$ for $d = -20, \dots, 20$ in stage $T - 1$:

<center><img src="img/last_b0d.png" width="70%"></center>

In [ ]:
# Transition probabilities from (B, 0, d) in stage T - 1
for d in range(-20, 21):
    dp.add_transition(T - 1, ('B', 0, d), 'none', ('B', 0, d), 1, 0)

## Boundary conditions

* Finally, the boundary conditions:

$$
f_T(n, k, d) = \begin{cases}
1 & \text{if } d > 0\\
r & \text{if } d = 0\\
0 & \text{if } d < 0
\end{cases}
\qquad \text{for } n \in \{\text{A}, \text{B}\}, k \in \{0, 3, 6\}, d = -20,\dots,20
$$

In [ ]:
# Boundary conditions


In [ ]:
# Solution
# Boundary conditions
for n in ['A', 'B']:
    for k in [0, 3, 6]:
        for d in range(-20, 21):
            if d > 0:
                dp.add_boundary(state=(n, k, d), value=1)
            elif d == 0:
                dp.add_boundary(state=(n, k, d), value=r)
            else:
                dp.add_boundary(state=(n, k, d), value=0)

## Solving the stochastic dynamic program

In [ ]:
# Solve the stochastic dynamic program
value, policy = dp.solve()

## Interpreting output from the stochastic dynamic program

**Example 1.** What is the maximum probability that Team A wins after scoring a touchdown in the first possession?

In [ ]:
# Solutions
print(value[0, ('A', 6, 6)])

**Example 2.** What should Team A do after scoring a touchdown in the first possession?

In [ ]:
# Solutions
policy[0, ('A', 6, 6)]

**Example 3.** Suppose Team A just scored a touchdown, making it 1 point behind. How does (1) the probability of Team A winning and (2) Team A's optimal strategy change depending on which possession this happened? Why do the trends you identified make sense?

- _Hint._ Write a `for` loop that prints out the information you want.

In [ ]:
# Solutions
d = -1
for t in range(number_of_stages - 1):
    print(f"Points ahead: {d}   Possession: {t}    Go for: {policy[t, ('A', 6, d)]}   Pr(win): {value[t, ('A', 6, d)]}")

**Example 4.** Redo Example 3, but under the assumption that Team A just scored a touchdown, making it 4 points ahead.

In [ ]:
# Solutions
d = 4
for t in range(number_of_stages - 1):
    print(f"Points ahead: {d}   Possession: {t}    Go for: {policy[t, ('A', 6, d)]}   Pr(win): {value[t, ('A', 6, d)]}")